In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.metric_utils import rmse_score
import pandas as pd
import numpy as np
import pickle

* Muti-output Model

In [2]:
with open(f'../Model/catboost_all_20220913235830.pickle', 'rb') as f:
    muti_output_model = pickle.load(f)

* Single output Model

In [3]:
with open(f'../Model/catboost_single_20220913235830.pickle', 'rb' ) as f:
    single_output_model_dic = pickle.load(f)

## load Data

In [4]:
with open(f'../Data/meta/train_x_after_feature_engineering.pickle', 'rb') as f:
    train_x_after_fe = pickle.load(f)

In [5]:
train_x_after_fe.head(3)

,clean_pressure31,clean_pressure33,clean_pressure41,clean_pressure51,clean_pressure52,clean_pressure61,clean_pressure71,clean_pressure72,clean_pressure81,clean_pressure91,...,clean_pressure82_group,oven_pb1_group,oven_pb2_group,oven_a1_group,oven_b1_group,oven_b3_group,painting_g3_act_hvv_group,painting_g8_act_f_air_group,painting_g9_act_hvc_group,env_rpi05_temp_group
0,-0.845269,-2.274242,2.327716,2.163639,-2.965910,1.903804,0.434963,2.902725,1.936083,-0.427078,...,5,5,3,2,2,1,3,0,4,1
1,-0.845888,-2.273734,2.327716,2.172853,-2.965910,1.903804,0.427070,2.906072,1.950879,-0.471062,...,5,5,3,2,2,1,3,0,4,1
4,-0.855656,-2.295590,2.327716,2.168243,-2.964846,1.903804,0.380329,2.902725,1.807915,-0.562123,...,5,5,3,2,2,1,3,0,1,1


In [6]:
with open(f'../Data/meta/train_y_after_feature_engineering.pickle', 'rb') as f:
    train_y_after_fe = pickle.load(f)

In [7]:
train_y_after_fe.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.875155,-1.366112,-1.107743,-1.662985,-0.752201,-2.795963
1,-3.261590,-1.215837,-2.053786,-1.706093,-1.202112,-2.974941
4,-1.715849,-0.539601,-1.896112,-1.231903,-1.202112,-1.602776


In [8]:
# You need to use original values of target variable to evaluate RMSE
with open(f'../Data/meta/non_scaled_train_y_after_feature_engineering.pickle', 'rb') as f:
    non_scaled_train_y = pickle.load(f)

In [9]:
non_scaled_train_y.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,42,56,82,41,73,45
1,37,60,58,40,64,42
4,57,78,62,51,64,65


In [10]:
# RandomForestRegressor(n_estimators=10, random_state=42)

In [11]:
stack_model_dic = {}
for col in train_y_after_fe.columns:
    estimators = [
        (f'{col}_model_1', single_output_model_dic[col]['model']),
        (f'{col}_model_2', single_output_model_dic[col]['model'])
    ]
    reg = StackingRegressor(
        estimators=estimators,
        final_estimator=LinearRegression()
    )

    reg.fit(train_x_after_fe, train_y_after_fe['sensor_point5_i_value'])
    stack_model_dic[col] = {'model': reg}

In [12]:
stack_model_dic[col]['model']

StackingRegressor(estimators=[('sensor_point10_i_value_model_1',
                               <catboost.core.CatBoostRegressor object at 0x0000010F10F99F70>),
                              ('sensor_point10_i_value_model_2',
                               <catboost.core.CatBoostRegressor object at 0x0000010F10F99F70>)],
                  final_estimator=LinearRegression())

In [13]:
with open(f'../Model/target_col_list.pickle', 'rb' ) as f:
    col_y_li = pickle.load(f)
array_li= [ stack_model_dic[col_y]['model'].predict(train_x_after_fe).reshape(-1,1) for col_y in col_y_li ]
all_res = np.concatenate(array_li, axis=1)

In [14]:
# Load y-scaler model
with open(f'../Model/standard_scaler_y.pickle', 'rb') as f:
    scaler_y = pickle.load(f)

In [15]:
predictions = scaler_y.inverse_transform(all_res)
targets = non_scaled_train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value      7.303188
sensor_point6_i_value     27.196275
sensor_point7_i_value     27.113154
sensor_point8_i_value     25.253559
sensor_point9_i_value     19.648987
sensor_point10_i_value    16.154748
dtype: float64
